## 통계 기반 기법 개선하기

## PMI : 점별 상호 정보량(Pointwise Mutual Information), log 함수
#### 높을수록 관련성이 높다

![image](https://t1.daumcdn.net/cfile/tistory/267AA340587EFB8737)

### PPMI(Positive PMI) : 양의 상호 정보량,  PPMI(x,y) = max(0,PMI(x,y))

In [1]:
from mynlp import preprocess, create_co_matrix, cos_similarity, most_similar
import numpy as np

In [3]:
text = 'You say goodbye and I say hello.' 
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)
print(word_to_id)
vocab_size = len(word_to_id)  # 7
C = create_co_matrix(corpus,vocab_size)
print('동시발생행렬:\n',C)
C.shape

[0 1 2 3 4 1 5 6]
{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}
동시발생행렬:
 [[0 1 0 0 0 0 0]
 [1 0 1 0 1 1 0]
 [0 1 0 1 0 0 0]
 [0 0 1 0 1 0 0]
 [0 1 0 1 0 0 0]
 [0 1 0 0 0 0 1]
 [0 0 0 0 0 1 0]]


(7, 7)

### PPMI 행렬

In [25]:
M =  np.zeros_like(C, dtype=np.float32) # (7,7)

N = np.sum(C)  # 14 , 동시발생행렬 요소전체의 합
print(N)

S = np.sum(C,axis=0) # 동시발생행렬 요소의 수직 방향 합
print(S)             # [1 4 2 2 2 2 1]

total = C.shape[0]*C.shape[1]  # 49, 동시발생행렬 요소의 총갯수

for i in range(C.shape[0]) : # 7 회
    for j in range(C.shape[1]) : # 7 회
        # C[i,j] : 동시발생행렬의 요소한개의 값, 비교하려는 두 단어의 동시발생횟수
        # S[0] : 동시발생행렬에서 열의 'you' 단어의 동시 발생횟수
        # S[i] : 동시발생행렬에서 열의 한 단어의 동시 발생횟수 , 비교하려는 두단어 중 하나 ,x
        # S[j] : 동시발생행렬에서 열 한 단어에 대해 비교하려는 나머지 단어들의 동시 발생횟수, y
        # print(i,j,C[i,j])
        # print('S[%d]:'%i,S[i],'S[%d]:'%j,S[j])
        
        pmi = np.log2(C[i,j]*N /(S[i]*S[j]) + 1e-8)
        M[i,j] = max(0,pmi)
        
print('PPMI:\n', M)   # PPMI 행렬로 변환 
print(C)

14
[1 4 2 2 2 2 1]
PPMI:
 [[0.        1.8073549 0.        0.        0.        0.        0.       ]
 [1.8073549 0.        0.8073549 0.        0.8073549 0.8073549 0.       ]
 [0.        0.8073549 0.        1.8073549 0.        0.        0.       ]
 [0.        0.        1.8073549 0.        1.8073549 0.        0.       ]
 [0.        0.8073549 0.        1.8073549 0.        0.        0.       ]
 [0.        0.8073549 0.        0.        0.        0.        2.807355 ]
 [0.        0.        0.        0.        0.        2.807355  0.       ]]
[[0 1 0 0 0 0 0]
 [1 0 1 0 1 1 0]
 [0 1 0 1 0 0 0]
 [0 0 1 0 1 0 0]
 [0 1 0 1 0 0 0]
 [0 1 0 0 0 0 1]
 [0 0 0 0 0 1 0]]


In [26]:
# PPMI 행렬 변환 함수 구현 
def ppmi(C,verbose=False, eps=1e-8):
    M =  np.zeros_like(C, dtype=np.float32) 
    N = np.sum(C)  
    S = np.sum(C,axis=0)
    total = C.shape[0]*C.shape[1]  

    for i in range(C.shape[0]) : 
        for j in range(C.shape[1]) :
            pmi = np.log2(C[i,j]*N /(S[i]*S[j]) + 1e-8)
            M[i,j] = max(0,pmi)
            
    return M 